# 1. 使用FUNCTION_CALL模式

In [14]:
from langchain.memory import ConversationBufferMemory
import os
import dotenv
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 提供提示词模板（以ChatPromptTemplate为例）
# 5. 定义提示词模板
prompt_template = ChatPromptTemplate.from_messages([
	("system", "你是一个有用的助手，可以回答问题并使用工具。"),
	("system", "{agent_scratchpad}"),
	("system", "{chat_history}"),  # 存储多轮对话的历史记录 如果你没有显式传入 chat_history，	Agent 会默认将其视为空列表[]
	("human", "{input}"),

])
# 记忆实例
memory = ConversationBufferMemory(
	return_messages=True,
	memory_key="chat_history"
)
# 获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True,
	memory=memory  # 记忆实例
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "查询一下长沙今天的天气情况"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `长沙今天的天气情况`


[{'title': '长沙空气污染：实时PM2.5空气质量指数（AQI）', 'url': 'https://aqicn.org/city/changsha/cn/m/', 'content': 'Air Quality Data provided by the Weather Underground (wunderground.com)  and the Hunan Environmental Protection Agency (湖南省环境保护厅) (sthjt.hunan.gov.cn)  and the China National Urban air quality real-time publishing platform (全国城市空气质量实时发布平台) (113.108.142.147:20035)\n\n气象数据\n\n长沙目前的天气状况在測量（星期三）23:00\n\n28°C\n\n湿度: 78%  \n空气压力：: 1007毫巴\n\n空气质量预报。\n\n37°C\n\n28°C\n\n37°C\n\n28°C\n\n37°C\n\n25°C\n\n30°C\n\n23°C\n\n33°C\n\n25°C\n\n34°C\n\n24°C\n\n29°C\n\n22°C [...] |  |  |  |  |  |\n ---  --- \n|  | 当前 | 过去5天内数据 | 最小 | 最大 |\n| PM2.5 | 65 |  |  |  |\n| PM10 |  |  |  |  |\n| NO2 |  |  |  |  |\n| SO2 |  |  | 1 |  |\n| CO | 6 |  | 2 | 6 |\n| 温度。 | 28 |  | 25 | 38 |\n| 露点   | 0 | 0 |\n| 空气压力 | 1007 |  | 1004 | 1012 |\n| 湿度 | 78 |  | 37 | 94 |\n| 风 | 1 |  | 0 | 10 |\n\nSHARE\n\n|  |  |\n --- |\n|  | 分享: 长沙空气质量指数 65 

In [15]:
# 清空记忆
# memory.clear()

# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "上海的呢？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `上海今天的天气情况`


[{'title': '10月10日上海天气预报晴到多云', 'url': 'http://sh.bendibao.com/news/20221010/260681.shtm', 'content': '上海中心气象台2022年10月10日11点钟发布今天和明天上海市天气预报：晴到多云，明天多云。西北风4-5级阵风6级，沿江沿海地区5级阵风6-7级，傍晚都转4-5级，明天北到东北风3-4级。今天最高温度18度，明天最低温度12度。今天相对湿度20%-80%。\n\n 上海已于10月5日入秋，今天的天气也转好了，晴到多云但西北风风力依然较大白天4~5级阵风6级傍晚起有所减小\n今天早间气温不高，全市大多10~13℃之间体感变冷，出行一定要注意添衣.\n\n白天虽有阳光助力，但气温回升缓慢，预计最高气温在18℃左右，午后暖阳体感会比较舒适\n\n本周秋高气爽的天气在线，以晴-多云-阴这样的云系变化为主，没有雨水打扰，是秋天该有的模样\n\n需要注意的是，周二早间比周一更冷，中心城区最低气温仅有11℃，郊区更低更冷，且这一天昼夜温差可接近10度\n\n在着装方面要多注意谨防感冒，其他时间气温整体较适中，保持在14~24℃之间\n\n手机访问上海本地宝首页\n\n相关推荐 上海天气上海天气预报 [...] 【导语】：申城今日晴到多云，西北风4-5级阵风6级，沿江沿海地区5级阵风6-7级，傍晚都转4-5级，今天最高温度18度。\n\n上海中心气象台2022年10月10日11点钟发布今天和明天上海市天气预报：晴到多云，明天多云。西北风4-5级阵风6级，沿江沿海地区5级阵风6-7级，傍晚都转4-5级，明天北到东北风3-4级。今天最高温度18度，明天最低温度12度。今天相对湿度20%-80%。\n\n 上海已于10月5日入秋，今天的天气也转好了，晴到多云但西北风风力依然较大白天4~5级阵风6级傍晚起有所减小\n今天早间气温不高，全市大多10~13℃之间体感变冷，出行一定要注意添衣.\n\n白天虽有阳光助力，但气温回升缓慢，预计最高气温在18℃左右，午后暖阳体感会比较舒适\n\n本周秋高气爽的天气在线，以晴-多云-

# 2. 使用ReAct模式

In [16]:
# 举例1
import os
import dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain import hub

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 提供提示词模板（使用langchain hub中官方ReAct提示词模板）
prompt_template = hub.pull("hwchase17/react-chat")
# 记忆实例
memory = ConversationBufferMemory(
	return_messages=True,
	memory_key="chat_history"
)
# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True,
	memory=memory
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "自动化行业发展前景怎么样？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search
Action Input: 自动化行业发展前景
```[{'title': '2024年中国工业自动化市场发展现状及未来发展前景', 'url': 'http://ksretech.com/newsDetail/394', 'content': '四、工业自动化发展前景\n\n1.工业自动化下游市场规模持续增长，反推工业自动化行业发展\n\n随着工业自动化技术的不断成熟，生产装置，控制装置、反馈装置和辅助装置等工业自动化设备在机床、风电、纺织、包装、塑料、建筑、采矿、交通运输和医疗等行业的应用规模不断扩大，使得工业自动化市场得到了长足的发展。\n\n2.智能化、高精度、高可靠性带来产品高端需求\n\n随着信息技术与制造技术的发展，以新型传感器、智能控制系统、工业机器人、自动化成套生产线等智能制造装备产业体系初步形成，自动化设备对其传动与制动系统的智能化、精度和可靠性提出了更高的要求。随着设备向智能化、自动化的发展，下游应用需求的变化将为工自动化行业带来更多的市场空间。\n\n3.国产化率快速提升 [...] 我国工业自动化行业起步较晚，但发展势头强劲。虽然我国工业自动化在关键的核心技术方面与外资品牌有显著差距，但近年来国家陆续颁布的一系列法规政策为我国工业自动化行业的发展提供了强劲的动力。中国工业自动化行业取得明显进步，国产替代进程加快。数据显示，2023年中国工业自动化市场规模达3115亿元，较2022年同比增长约11%。中商产业研究院分析师预测，2024年中国工业自动化市场规模将增至3531亿元\n\n数据来源：中商产业研究院\n\n未来随着全球工业4.0时代的持续推进，各应用领域对工业自动化设备的需求将进一步增加，中商产业研究院分析师预测，2024年全球工业自动化市场规模到将达5095.9亿美元。\n\n数据来源：Frost&Sullivan\n\n2.工业自动化核心产品生产情况 [...] 随着近年来全球制造业向中国转移，工业自动化行业的中国企业获取了先进的技术和制造经验，整体行业发展较快，但由于我国核心技术和高端产品对外依存度仍然较高，国内企业仍与发达

In [17]:
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "新零售呢？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search
Action Input: "新零售 发展前景 2024"
```
[{'title': '2024年電子商務與新零售的未來趨勢 - Searchingineer.com', 'url': 'https://searchingineer.com/blog/2024%E5%B9%B4%E9%9B%BB%E5%AD%90%E5%95%86%E5%8B%99%E8%88%87%E6%96%B0%E9%9B%B6%E5%94%AE%E7%9A%84%E6%9C%AA%E4%BE%86%E8%B6%A8%E5%8B%A2/', 'content': '在快速變化的商業環境中，電子商務和新零售正迎來前所未有的機遇與挑戰。本文將探討2024年最重要的三大趨勢：虛實零售整合、AI助力零售業、以及零售產業的永續環保發展。讓我們深入了解這些趨勢如何塑造未來的零售格局。\n\n### 趨勢一：虛實零售整合\n\n#### 問題\n\n傳統零售商正面臨線上競爭的巨大壓力，而純電商品牌也意識到實體存在的重要性。如何在這兩個領域中取得平衡，成為當前零售業的一大挑戰。\n\n#### 激化\n\n根據麥肯錫的預測，到2025年，電子商務零售業務將佔全球零售收入的一半。這意味著不適應這種趨勢的企業可能會失去大量市場份額。同時，研究顯示，60%的Z世代消費者仍偏好在實體店購物，這凸顯了線下體驗的持續重要性。\n\n#### 解決方案\n\n##### 線上品牌的實體布局\n\n 案例分析：美而快（PAZZO）的成功轉型\n 在實體消費的客戶中，72%的人在30天內會回到線上消費，遠高於普通線上會員24%的月回購率。\n\n實施策略：\n\n1. 選擇合適的地理位置，優先考慮年輕消費群體集中的商圈。\n2. 設計獨特的店內體驗，結合線上特色與線下互動。\n3. 利用實體店收集的數據來優化線上運營。 [...] #### 行動建議\n\n審視你的供應鏈和產品線，找出可以改進的環節。考慮推出環保產品線或者實施回收計劃，並將這些努力透明地傳達給消費者。\n\n### 結語\n\n2024年的零售業正面臨

In [27]:
# 举例2：自定义提示词模板
import os
import dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain import hub

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 自定义提示词模版
# 自定义提示词模版
template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""
prompt_template = PromptTemplate(
	template=template
)
# 记忆实例
memory = ConversationBufferMemory(
	return_messages=True,
	memory_key="chat_history"
)
# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True,
	memory=memory,
	handle_parsing_errors=True  # 处理解析错误
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "2025年自动化行业发展前景怎么样？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search
Action Input: 2025年自动化行业发展前景
```[{'title': '2025年自动化机器人行业发展现状及未来发展趋势深度分析', 'url': 'https://www.qyresearch.com.cn/news/25727/automated-robots', 'content': 'QYResearch调研显示，2024年全球云自动化机器学习市场规模大约为 亿美元，预计2031年将达到 亿美元，2025-2031期间年复合增长率（CAGR）为 %。未来几年，本行业具有很大不确定性，本文的2025-2031年的预测数据是基于过去几年的历史发展、行业专家观点、以及本文分析师观点，综合给出的预测。\n\n出版时间 : 2025-08-01行业 : 机械及设备\n\n根据QYResearch（北京恒州博智国际信息咨询有限公司）的统计及预测，2024年全球包装自动化机器人市场销售额达到了35.51亿美元，预计2031年将达到75.86亿美元，年复合增长率（CAGR）为10.7%（2025-2031）。地区层面来看，中国市场在过去几年变化较快，2024年市场规模为 百万美元，约占全球的 %，预计2031年将达到 百万美元，届时全球占比将达到 %。\n\n更多资讯 [...] QYResearch调研显示，2024年全球输液自动化机器人市场规模大约为 亿美元，预计2031年将达到 亿美元，2025-2031期间年复合增长率（CAGR）为 %。由于美国2025年关税框架的潜在转向已引发全球市场重大波动风险，本报告将深入评估最新关税调整及各国应对战略对输液自动化机器人市场竞争态势、区域经济联动及供应链重构的潜在影响。未来几年，本行业具有很大不确定性，本文的2025-2031年的预测数据是基于过去几年的历史发展、行业专家观点、以及本文分析师观点，综合给出的预测。\n\n出版时间 : 2025-10-03行业 : 软件及商业服务\n\n根据QYResearch报告出版商调研统计，2031年全球云自动化机器学习市场销售额预计将达到 亿元，年复合增长率（CAGR

In [28]:
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "新零售呢？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search
Action Input: "2025年新零售行业发展前景"
```[{'title': '2025年全球零售业25大趋势预测 - 亿邦动力', 'url': 'https://m.ebrun.com/570355.html', 'content': '2025年，随着技术进步和监管政策的变化，网络安全领域将迎来重大转变。网络安全专家预测，焦点之一将是AI应用的安全性，尤其是在AI技术已深度融入零售企业的背景下。2025年将出现更严格的隐私法规、更多的消费者数据控制权，以及对安全和道德数据使用的更高要求。零售商需要在个性化体验所需的数据获取与保护消费者隐私的责任之间寻找平衡。在不断发展的零售环境中，透明度、安全性和以隐私为先的策略将成为维护消费者信任的关键。\n\n注：文/联商网编辑部，文章来源：新零售(公众号ID：ixinlingshou)，本文为作者独立观点，不代表亿邦动力立场。\n\n文章来源：新零售\n\n单篇购买;)\n\n广告\n\n微信\n\n朋友圈\n\n2\n+1 [...] 2025年，零售技术创新的重点将转向提升员工体验。随着AI和自动化简化运营流程，零售企业将更注重为员工提供激发创造力、提高参与度和促进成长的工具。通过优化工作环境和培育持续学习的文化，零售商将提升生产效率，降低员工流失率。这一转变将推动业务成功，确保在技术演进中保持零售业的人文本质。\n\n8. 实体零售迎来“新生”\n\n实体店在2025年将继续焕发新机。作为提升客户留存、获取新客户、塑造品牌形象和培养忠诚度的基础，当代实体店必须提供无法在线上复制的沉浸式体验，展现品牌独特个性。这些空间通过真实互动建立信任和忠诚度，培养个人连接，并在品牌周围建立社群。在消费者寻求有意义连接的趋势下，实体店正好提供了理想的环境。\n\n9. 主流品牌加码店内服务体验\n\n越来越多的主流品牌将借鉴精品店和奢侈品店的经验，加大对店内服务和体验的投资。这些服务不仅是锦上添花，更是新的收入增长点，全球丝芙兰(Sephora)和Rituals门店的成功就是明证。最重要的是，店内服务是区分线上线下